In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np 
import time 
import numpy as np
import torch
import torch.nn as nn
import time
import gym 
import pickle 
from replay_memory import Memory
import argparse
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions.transformed_distribution import TransformedDistribution
from torch.distributions.transforms import TanhTransform
import tqdm

from nets import Actor 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
env_name='Ant-v3' 

In [5]:
data_path = "expert_data/Ant-v3_10_3765.pkl"  

with open(data_path, "rb") as f:
    data_good = pickle.load(f)
print('expert data loaded')

# data_good = data_good[:10]

expert data loaded


In [6]:
good_obs=[]
good_acts=[]
good_rews=[]

for traj in data_good: 
    s,a,r=traj  

    good_obs.append(s)
    good_acts.append(a)
    good_rews.append(r)

good_obs=np.vstack(good_obs)
good_acts=np.vstack(good_acts)
good_rews=np.vstack(good_rews) 

In [7]:
good_obs.shape, good_acts.shape, good_rews.shape

((10000, 111), (10000, 8), (10000, 1))

In [8]:
memory=Memory(maxlen=1000000)
memory.add_data(good_obs, good_acts, good_rews)

In [9]:
seed=0
torch.manual_seed(seed)
np.random.seed(seed) 
env = gym.make(env_name) 

ob_dim = env.observation_space.shape[0]
ac_dim = env.action_space.shape[0]

In [10]:
actor = Actor(ac_dim, ob_dim).to(device)

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(actor.parameters(), 0.005)

In [11]:
# predicted_actions.shape, batch_actions.shape

In [12]:
verbose=True
batch_size=100
training_timestep=20_000
for train_step in range(1, training_timestep+1):  
    batch_states, batch_actions, batch_rewards=memory.sample_batch(batch_size)  
    batch_states = torch.Tensor(batch_states).to(device)
    batch_actions=torch.Tensor(batch_actions).to(device)
    
    predicted_actions = actor.forward(batch_states)
    optimizer.zero_grad()  
    loss =loss_func(predicted_actions, batch_actions)
    loss.backward()
    optimizer.step()

    if verbose and (train_step % (training_timestep/10)==0):
        print('step:', train_step, ' loss', loss.item()) 

print('bc training completed')

step: 2000  loss 0.005139579530805349
step: 4000  loss 0.004350221715867519
step: 6000  loss 0.003830575617030263
step: 8000  loss 0.003945298492908478
step: 10000  loss 0.003657997353002429
step: 12000  loss 0.003890473395586014
step: 14000  loss 0.0030245971865952015
step: 16000  loss 0.0034294857177883387
step: 18000  loss 0.003278973512351513
step: 20000  loss 0.0030634175054728985
bc training completed


In [13]:
def play(env, policy, is_close=True, is_render=True, max_step=1000): 
    obs,info = env.reset()
    dones=False
    total_r=0
    step=0
    while not dones: 
        step+=1
        obs=torch.Tensor(obs[None]).to(device)
        ac = policy(obs)
        action=ac.cpu().detach().numpy() 

        obs, rewards, done, s, info = env.step(action.ravel())
        total_r +=rewards  
        if done:
            break
        if step>max_step:
            # print('max step reached')
            break
        # elif s:
        #     print('solved!')
        #     break
    if is_close:
        env.close()
    return {'reward':total_r, 'step':step-1}

In [14]:
env = gym.make(env_name)
play(env, actor, is_close=True, is_render=False)

{'reward': 4139.687880579301, 'step': 1000}

In [15]:
rewards=[]
for i in tqdm.tqdm( range(50) ):
    stats=play(env, actor, is_close=True, is_render=False)
    rewards.append(stats['reward'])
    # print(f'i={i} reward={stats["reward"]} step={stats["step"]}')
print(f'mean reward={np.mean(rewards) :.2f} std={np.std(rewards):.2f}')

100%|██████████| 50/50 [00:36<00:00,  1.36it/s]

mean reward=3828.40 std=826.02


In [17]:
print(f'mean reward={np.mean(rewards) :.2f} std={np.std(rewards):.2f}')

mean reward=3828.40 std=826.02
